

## HARSH ASHOK VALAMBE

## 20BRS1197
Basic Neural Network for identifying handwritten numbers (0-9) Recognition


In [ ]:
from keras import models
from keras import layers
from keras.datasets import mnist
import pandas as pd
from keras.utils import to_categorical

In [ ]:
#loading data 
(train_images, train_labels) , (test_images, test_labels) = mnist.load_data()

In [ ]:
#PRINTING THE AVAILABLE INFORMATION
print(train_images.shape)
print(train_labels.shape)

print(test_images.shape)
print(test_labels.shape)



(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [ ]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)
print(train_labels.shape)

(60000, 10)


In [ ]:
network = models.Sequential()
network.add(layers.Dense(512, activation="relu", input_shape=(28*28,)))
network.add(layers.Dense(10, activation="softmax"))

network.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=['accuracy'])

In [ ]:
train_images = train_images.reshape((60000, 28*28))
train_images = train_images.astype('float32')/255

test_images = test_images.reshape((10000, 28*28))
test_images = test_images.astype('float32')/255


In [ ]:
network.fit(train_images, train_labels, epochs=7, batch_size=128)

Epoch 1/7
469/469 [==============================] - 6s 11ms/step - loss: 0.2604 - accuracy: 0.9246
Epoch 2/7
469/469 [==============================] - 5s 11ms/step - loss: 0.1042 - accuracy: 0.9696
Epoch 3/7
469/469 [==============================] - 5s 11ms/step - loss: 0.0681 - accuracy: 0.9796
Epoch 4/7
469/469 [==============================] - 5s 11ms/step - loss: 0.0501 - accuracy: 0.9851
Epoch 5/7
469/469 [==============================] - 5s 12ms/step - loss: 0.0377 - accuracy: 0.9887
Epoch 6/7
469/469 [==============================] - 5s 11ms/step - loss: 0.0289 - accuracy: 0.9911
Epoch 7/7
469/469 [==============================] - 5s 11ms/step - loss: 0.0219 - accuracy: 0.9935


In [ ]:
test_loss, test_acc = network.evaluate(test_images, test_labels)

print(test_loss)
print("Accuracy=")
print(test_acc)

313/313 [==============================] - 1s 3ms/step - loss: 0.0639 - accuracy: 0.9827
0.06387661397457123
Accuracy=
0.982699990272522


# Implementing Deep Neural Network .

In [ ]:
"""
network.py
~~~~~~~~~~
IT WORKS
A module to implement the stochastic gradient descent learning
algorithm for a feedforward neural network.  Gradients are calculated
using backpropagation.  Note that I have focused on making the code
simple, easily readable, and easily modifiable.  It is not optimized,
and omits many desirable features.
"""

#### Libraries
# Standard library
import random

# Third-party libraries
import numpy as np

class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        #used zip to iterate b and w at the same phase
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""

        training_data = list(training_data)
        n = len(training_data)

        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {} : {} / {}".format(j,self.evaluate(test_data),n_test))
            else:
                print("Epoch {} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        
        
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

##### Define the loader class to load data

Here a local copy of mnist data is used.

The data is loaded using **Pickle** API and returned as training data, validation data and test data respectively.

In [ ]:
import pickle
import gzip

# Third-party libraries
import numpy as np

def load_data():
    """Return the MNIST data as a tuple containing the training data,
    the validation data, and the test data.
    The ``training_data`` is returned as a tuple with two entries.
    The first entry contains the actual training images.  This is a
    numpy ndarray with 50,000 entries.  Each entry is, in turn, a
    numpy ndarray with 784 values, representing the 28 * 28 = 784
    pixels in a single MNIST image.
    The second entry in the ``training_data`` tuple is a numpy ndarray
    containing 50,000 entries.  Those entries are just the digit
    values (0...9) for the corresponding images contained in the first
    entry of the tuple.
    The ``validation_data`` and ``test_data`` are similar, except
    each contains only 10,000 images.
    This is a nice data format, but for use in neural networks it's
    helpful to modify the format of the ``training_data`` a little.
    That's done in the wrapper function ``load_data_wrapper()``, see
    below.
    """
    f = gzip.open('/content/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    """Return a tuple containing ``(training_data, validation_data,
    test_data)``. Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.
    In particular, ``training_data`` is a list containing 50,000
    2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray
    containing the input image.  ``y`` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for ``x``.
    ``validation_data`` and ``test_data`` are lists containing 10,000
    2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional
    numpy.ndarry containing the input image, and ``y`` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to ``x``.
    Obviously, this means we're using slightly different formats for
    the training data and the validation / test data.  These formats
    turn out to be the most convenient for use in our neural network
    code."""
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

## LOADING THE MNIST.PKL.GZ 

In [ ]:
training_data, validation_data, test_data = load_data_wrapper()
training_data = list(training_data)

##### Training the network using Stochastic Gradient Descent algorithm.

In [ ]:
network = Network([784, 30,10])
network.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0 : 9037 / 10000
Epoch 1 : 9170 / 10000
Epoch 2 : 9272 / 10000
Epoch 3 : 9345 / 10000
Epoch 4 : 9328 / 10000
Epoch 5 : 9365 / 10000
Epoch 6 : 9426 / 10000
Epoch 7 : 9405 / 10000
Epoch 8 : 9405 / 10000
Epoch 9 : 9444 / 10000
Epoch 10 : 9447 / 10000
Epoch 11 : 9430 / 10000
Epoch 12 : 9475 / 10000
Epoch 13 : 9456 / 10000
Epoch 14 : 9446 / 10000
Epoch 15 : 9458 / 10000
Epoch 16 : 9504 / 10000
Epoch 17 : 9467 / 10000
Epoch 18 : 9489 / 10000
Epoch 19 : 9488 / 10000
Epoch 20 : 9496 / 10000
Epoch 21 : 9511 / 10000
Epoch 22 : 9515 / 10000
Epoch 23 : 9530 / 10000
Epoch 24 : 9506 / 10000
Epoch 25 : 9497 / 10000
Epoch 26 : 9525 / 10000
Epoch 27 : 9502 / 10000
Epoch 28 : 9509 / 10000
Epoch 29 : 9507 / 10000


**_Conclusion_**:

94.98% accuracy is obtained with neural network implemented. This network has been implemented using Stochastic Gradient Descent algorithm that minimizes the cost function. This algorithm implmented uses batching procedure for easier and faster processing of data.

# QUESTIONS:

# 1->CHANGE THE LEARNING RATE (VERY LOW,VERY HIGH)

# 2-> CHANGE NUMBER OF NEURONS IN THE HIDDEN LAYERS

# 3->REMOVE HIDDEN LAYER AND CHECK ACCURACY

In [ ]:
network1 = Network([784, 30,10])
network1.SGD(training_data, 30, 10, 0.5, test_data=test_data)

Epoch 0 : 7521 / 10000
Epoch 1 : 8683 / 10000
Epoch 2 : 8935 / 10000
Epoch 3 : 9020 / 10000
Epoch 4 : 9106 / 10000
Epoch 5 : 9153 / 10000
Epoch 6 : 9194 / 10000
Epoch 7 : 9205 / 10000
Epoch 8 : 9227 / 10000
Epoch 9 : 9253 / 10000
Epoch 10 : 9262 / 10000
Epoch 11 : 9286 / 10000
Epoch 12 : 9294 / 10000
Epoch 13 : 9315 / 10000
Epoch 14 : 9321 / 10000
Epoch 15 : 9319 / 10000
Epoch 16 : 9335 / 10000
Epoch 17 : 9342 / 10000
Epoch 18 : 9326 / 10000
Epoch 19 : 9354 / 10000
Epoch 20 : 9362 / 10000
Epoch 21 : 9362 / 10000
Epoch 22 : 9365 / 10000
Epoch 23 : 9365 / 10000
Epoch 24 : 9371 / 10000
Epoch 25 : 9378 / 10000
Epoch 26 : 9376 / 10000
Epoch 27 : 9379 / 10000
Epoch 28 : 9383 / 10000
Epoch 29 : 9389 / 10000


## AT LEARNING RATE OF 0.5 THERE IS AN ACCURACY OF 93.89%

In [ ]:
network2 = Network([784, 30,10])
network2.SGD(training_data, 30, 10, 7.0, test_data=test_data)

Epoch 0 : 9095 / 10000
Epoch 1 : 9154 / 10000
Epoch 2 : 9184 / 10000
Epoch 3 : 9269 / 10000
Epoch 4 : 9303 / 10000
Epoch 5 : 9360 / 10000
Epoch 6 : 9360 / 10000
Epoch 7 : 9367 / 10000
Epoch 8 : 9385 / 10000
Epoch 9 : 9442 / 10000
Epoch 10 : 9440 / 10000
Epoch 11 : 9466 / 10000
Epoch 12 : 9413 / 10000
Epoch 13 : 9447 / 10000
Epoch 14 : 9449 / 10000
Epoch 15 : 9410 / 10000
Epoch 16 : 9386 / 10000
Epoch 17 : 9452 / 10000
Epoch 18 : 9489 / 10000
Epoch 19 : 9447 / 10000
Epoch 20 : 9475 / 10000
Epoch 21 : 9473 / 10000
Epoch 22 : 9493 / 10000
Epoch 23 : 9495 / 10000
Epoch 24 : 9471 / 10000
Epoch 25 : 9513 / 10000
Epoch 26 : 9497 / 10000
Epoch 27 : 9512 / 10000
Epoch 28 : 9482 / 10000
Epoch 29 : 9487 / 10000


## ## AT LEARNING RATE OF 7.0 THERE IS AN ACCURACY OF 94.87%

In [ ]:
network3 = Network([784, 50,10])
network3.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0 : 9113 / 10000
Epoch 1 : 9284 / 10000
Epoch 2 : 9348 / 10000
Epoch 3 : 9420 / 10000
Epoch 4 : 9430 / 10000
Epoch 5 : 9496 / 10000
Epoch 6 : 9481 / 10000
Epoch 7 : 9507 / 10000
Epoch 8 : 9510 / 10000
Epoch 9 : 9518 / 10000
Epoch 10 : 9515 / 10000
Epoch 11 : 9549 / 10000
Epoch 12 : 9547 / 10000
Epoch 13 : 9580 / 10000
Epoch 14 : 9557 / 10000
Epoch 15 : 9561 / 10000
Epoch 16 : 9573 / 10000
Epoch 17 : 9580 / 10000
Epoch 18 : 9591 / 10000
Epoch 19 : 9570 / 10000
Epoch 20 : 9552 / 10000
Epoch 21 : 9575 / 10000
Epoch 22 : 9579 / 10000
Epoch 23 : 9584 / 10000
Epoch 24 : 9606 / 10000
Epoch 25 : 9604 / 10000
Epoch 26 : 9578 / 10000
Epoch 27 : 9589 / 10000
Epoch 28 : 9584 / 10000
Epoch 29 : 9600 / 10000


### CHANGING NUMBER OF NEURONS IN HIDDEN LAYER TO 50 results for accuracy being  96%

In [ ]:
network4 = Network([784,30,10])
network4.SGD(training_data, 20, 10, 7.0, test_data=test_data)

Epoch 0 : 9050 / 10000
Epoch 1 : 9242 / 10000
Epoch 2 : 9317 / 10000
Epoch 3 : 9326 / 10000
Epoch 4 : 9374 / 10000
Epoch 5 : 9359 / 10000
Epoch 6 : 9384 / 10000
Epoch 7 : 9407 / 10000
Epoch 8 : 9426 / 10000
Epoch 9 : 9472 / 10000
Epoch 10 : 9453 / 10000
Epoch 11 : 9466 / 10000
Epoch 12 : 9439 / 10000
Epoch 13 : 9482 / 10000
Epoch 14 : 9417 / 10000
Epoch 15 : 9453 / 10000
Epoch 16 : 9470 / 10000
Epoch 17 : 9429 / 10000
Epoch 18 : 9476 / 10000
Epoch 19 : 9495 / 10000


### CHANGING NUMBER OF NEURONS IN HIDDEN LAYER TO 30 results in reducing the accuracy to 94.95%

In [ ]:
network5 = Network([784,10])
network5.SGD(training_data, 20, 10, 3.0, test_data=test_data)

Epoch 0 : 6528 / 10000
Epoch 1 : 6598 / 10000
Epoch 2 : 6592 / 10000
Epoch 3 : 6632 / 10000
Epoch 4 : 6643 / 10000
Epoch 5 : 6606 / 10000
Epoch 6 : 6634 / 10000
Epoch 7 : 6642 / 10000
Epoch 8 : 6649 / 10000
Epoch 9 : 6671 / 10000
Epoch 10 : 6666 / 10000
Epoch 11 : 7482 / 10000
Epoch 12 : 7489 / 10000
Epoch 13 : 7506 / 10000
Epoch 14 : 7478 / 10000
Epoch 15 : 7497 / 10000
Epoch 16 : 7520 / 10000
Epoch 17 : 7515 / 10000
Epoch 18 : 7538 / 10000
Epoch 19 : 7522 / 10000


### After removing the hidden layer the accuracy reduces to 75.22%

In [ ]:
network6 = Network([784,50,30,10])
network6.SGD(training_data, 20, 10, 3.0, test_data=test_data) 

Epoch 0 : 9101 / 10000
Epoch 1 : 9292 / 10000
Epoch 2 : 9361 / 10000
Epoch 3 : 9322 / 10000
Epoch 4 : 9460 / 10000
Epoch 5 : 9425 / 10000
Epoch 6 : 9487 / 10000
Epoch 7 : 9426 / 10000
Epoch 8 : 9509 / 10000
Epoch 9 : 9502 / 10000
Epoch 10 : 9504 / 10000
Epoch 11 : 9508 / 10000
Epoch 12 : 9500 / 10000
Epoch 13 : 9480 / 10000
Epoch 14 : 9545 / 10000
Epoch 15 : 9547 / 10000
Epoch 16 : 9559 / 10000
Epoch 17 : 9483 / 10000
Epoch 18 : 9562 / 10000
Epoch 19 : 9563 / 10000


### ADDING ONE MORE LAYER GIVES A 95.63% ACCURACY